This is largely a set of tests of the different ways indexing can interact with other primatives.

In [1]:
from pangolin import Given, d, t, I, IID, vmap, sample, E, P, var, std, cov, corr, makerv, jags_code
from matplotlib import pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{:6.2f}'.format}) # print nicely

In [2]:
# Thoroughly test 1-D indexing (no random variables)
X0 = np.arange(10)/10
X = makerv(X0)
print('E(X[0])')
print(E(X[0]))
print(X0[0])
# print('E(X[-2])')
# print(E(X[-2]))
# print(X0[-2])
print('E(X[:])')
print(E(X[:]))
print(X0[:])
print('E(X[2:5])')
print(E(X[2:5]))
print(X0[2:5])
print('E(X[9:3:-2])')
print(E(X[9:3:-2]))
print(X0[9:3:-2])

E(X[0])
0.0
0.0
E(X[:])
[  0.00   0.10   0.20   0.30   0.40   0.50   0.60   0.70   0.80   0.90]
[  0.00   0.10   0.20   0.30   0.40   0.50   0.60   0.70   0.80   0.90]
E(X[2:5])
[  0.20   0.30   0.40]
[  0.20   0.30   0.40]
E(X[9:3:-2])
[  0.90   0.70   0.50]
[  0.90   0.70   0.50]


In [3]:
#x = d.norm(0,1)
#y = IID(d.norm(x,1),4)
x = d.norm(0,1)
y = vmap(d.norm)(x*[1,2,3,4,5],[1,1,1,1,1])
o = np.array([9,2,-7,3,4]) # observations

#print('E(x|y0=o0)        ',E(x,Given(y[0]==o[0])))
#print('E(x|y2=o2)        ',E(x,Given(y[2]==o[2])))
print('each group should be equal')
print('---')
print('E(x|y0=o0,y2=o2)       ',E(x,Given(y[0]==o[0],y[2]==o[2])))
print('E(x|y[(0,2)]=o[(0,2)]) ',E(x,Given(y[[0,2]]==o[[0,2]])))
print('---')
print('E(x|y[0:2]=o[0:2])     ',E(x,Given(y[0:2]==o[0:2])))
print('E(x|y[[0,1]]=o[[0,1]]) ',E(x,Given(y[[0,1]]==o[[0,1]])))
print('---')
print('E(x|y[0:2]=o[0:2])     ',E(x,Given(y[0:2]==o[0:2])))
print('E(x|y[[0,1]]=o[[0,1]]) ',E(x,Given(y[[0,1]]==o[[0,1]])))
print('---')
print('E(x|y[::-2],o[::-2])   ',E(x,Given(y[::-2]==o[::-2])))
print('E(x|y4=o4,y2=o2,y0=o0) ',E(x,Given(y[4]==o[4],y[2]==o[2],y[0]==o[0])))

each group should be equal
---
E(x|y0=o0,y2=o2)        -1.0843617375
E(x|y[(0,2)]=o[(0,2)])  -1.0843617375
---
E(x|y[0:2]=o[0:2])      2.175531537
E(x|y[[0,1]]=o[[0,1]])  2.175531537
---
E(x|y[0:2]=o[0:2])      2.175531537
E(x|y[[0,1]]=o[[0,1]])  2.175531537
---
E(x|y[::-2],o[::-2])    0.225841342434
E(x|y4=o4,y2=o2,y0=o0)  0.225841342434


In [4]:
# tracing and indexing
def f(a,b):
    return a[1]*b[2]
g = vmap(f,[True,True])
A = makerv([[0,2,0],[0,3,0]])
B = makerv([[0,0,4],[0,0,5]])
print('E(g(A,B))')
print(E(g(A,B)))
print('expected')
print(A.value[:,1]*B.value[:,2])

E(g(A,B))
[  8.00  15.00]
expected
[ 8 15]


In [5]:
# tracing and indexing
def f(a,b,n,m):
    return a[n]*b[m]
g = vmap(f,[True,True,True,True])
A0 = np.array([[1.,0,0],[0,0,2.]])
B0 = np.array([[0,0,3.],[0,4.,0]])
N0 = np.array([0,2])
M0 = np.array([2,1])
A = makerv(A0)
B = makerv(B0)
N = makerv(N0)
M = makerv(M0)
print('E(g(A,B,N,M))',E(g(A,B,N,M)))
print('expected     ',[f(a,b,n,m) for (a,b,n,m) in zip(A0,B0,N0,M0)])
print('expected     ',[A0[0,N0[0]]*B0[0,M0[0]],A0[1,N0[1]]*B0[1,M0[1]]])

E(g(A,B,N,M)) [  3.00   8.00]
expected      [3.0, 8.0]
expected      [3.0, 8.0]


In [6]:
# partial vectorization
def f(a,b,n,m):
    return a[n]*b[m]
g = vmap(f,[True,False,False,True])
A0 = np.array([[0,0,1.],[0,0,2.]])
B0 = np.array([3.,4.,0.])
N0 = np.array(2)
M0 = np.array([1,0])
A = makerv(A0)
B = makerv(B0)
N = makerv(N0)
M = makerv(M0)
print('E(g(A,B,N,M))',E(g(A,B,N,M)))
print('expected     ',[f(a,B0,N0,m) for (a,m) in zip(A0,M0)])
print('expected     ',[A0[0,N0]*B0[M0[0]],A0[1,N0]*B0[M0[1]]])

E(g(A,B,N,M)) [  4.00   6.00]
expected      [4.0, 6.0]
expected      [4.0, 6.0]


In [7]:
# make an outer product via vectorization
outer = vmap(vmap(lambda a,b:a*b,[False,True]),[True,False])
x = np.array([1,2,3])
y = np.array([4,5,6])
print('E(outer(x,y))')
print(E(outer(x,y)))
print('np.outer(x,y)')
print(np.outer(x,y))

E(outer(x,y))
[[  4.00   5.00   6.00]
 [  8.00  10.00  12.00]
 [ 12.00  15.00  18.00]]
np.outer(x,y)
[[ 4  5  6]
 [ 8 10 12]
 [12 15 18]]
